<a href="https://colab.research.google.com/github/tomonari-masada/course2025-stats2/blob/main/07_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 線形回帰
* ベイズ線形回帰モデルを実装する。
* それだけでなく、外れ値に強いロバストな回帰分析を行なう方法も示す。

* 下のWebページを参考にしました。
  * https://bambinos.github.io/bambi/notebooks/t_regression.html

* ベイズ線形回帰については、下の記事も参考になります。
 * Part 1 https://towardsdatascience.com/applied-bayesian-inference-pt-1-322b25093f62
 * Part 2 https://towardsdatascience.com/applied-bayesian-inference-with-python-pt-2-80bcd63b507e

## 準備

In [ ]:
!pip install numpyro

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import jax
import jax.numpy as jnp
from jax import random
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC, Predictive

import arviz as az

%config InlineBackend.figure_format = 'retina'

plt.rcParams.update({'figure.constrained_layout.use':True})
plt.style.use("bmh")
az.style.use("arviz-darkgrid")

rng_key = random.PRNGKey(0)

numpyro.set_platform("cpu")

## ベイズ線形回帰

### 復習：最尤推定
* 線形回帰は、以下のように定式化できる。
$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \cdots + \beta_d X_d + \epsilon $$
 * $\beta_0, \beta_1, \ldots, \beta_d$が推定すべき係数。
 * $\beta = (\beta_0, \beta_1, \ldots, \beta_d)$、$X = (1, X_1, \ldots, X_d)$とおくと、上式は以下のように書ける。
$$ Y = \beta^\top X + \epsilon $$
* 最小二乗法では、誤差項$\epsilon$が正規分布に従うと仮定し、最尤推定する。
* $i$番目の訓練データの説明変数の値を$x_i = (1, x_{i,1}, \ldots, x_{i,d})$、対応する目的変数の値を$y_i$と書くことにすると、データ尤度は
$$ L(\beta; \{ (x_1,y_1), \ldots, (x_N,y_N) \} ) = \prod_{i=1}^N \frac{1}{\sqrt{2\pi\sigma^2}} \exp \bigg( - \frac{ ( y_i - \beta^\top x_i )^2 }{2 \sigma^2} \bigg) $$
* $L(\beta; \{ (x_1,y_1), \ldots, (x_N,y_N) \} )$を最大にする$\hat{\beta}$ で答えるのが、最小二乗法。
 * $\sigma$を定数とみなすならば、$\sigma$がいくらであろうと、答えは同じ。

### ベイズ推論
* やはり、誤差項が正規分布に従うと仮定する。
$$ \epsilon \sim N(0, \sigma^2)$$
* これを書き直すと
$$ Y \sim N(\beta^\top X, \sigma^2) $$
* ベイズ推論を使う場合、$\beta$や$\sigma$が従う分布を、事前分布として導入する。
* そして、$\beta$や$\sigma$が従う事後分布を求める。

## 通常のベイズ線形回帰
* 今回は説明変数が一つだけの場合を考える。
* ここで「通常の」とは、誤差項が正規分布に従うと仮定する、という意味。
* つまり、
$$ Y \sim N(\beta_0 + \beta_1 X_1, \sigma^2) $$
* そして、$\beta_0$, $\beta_1$, $\sigma$について事前分布を導入する。

### 人工的なデータ集合を生成
* 真の切片$\beta_0$は$1$、真の傾き$\beta_1$は$2$であるとして、データを人工的に生成する。
 * 誤差項が従う正規分布の標準偏差$\sigma$は0.5と設定している。

In [ ]:
size = 200
true_intercept = 1
true_slope = 2

x = jnp.linspace(0, 1, size)
# y = a + b*x
true_regression_line = true_intercept + true_slope * x
# add noise
rng_key, rng_key_ = random.split(rng_key)
y = true_regression_line + random.normal(rng_key_, shape=(size,)) * 0.5

data = dict(x=x, y=y)

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111, xlabel="x", ylabel="y")
ax.plot(x, y, "x", label="data")
ax.plot(x, true_regression_line, label="true regression line", c="r")
plt.legend(loc=0);

### モデルの指定
* 切片$\beta_0$と傾き$\beta_1$の事前分布は、平均$0$、標準偏差$20$の正規分布とする。
$$
\begin{align}
\beta_0 & \sim N(0, 20^2) \\
\beta_1 & \sim N(0, 20^2)
\end{align}
$$
* 誤差項$\epsilon \sim N(0, \sigma^2)$の標準偏差$\sigma$は、half Cauchy分布に従うとする。
 * scaleパラメータは10とする。
$$\sigma \sim \text{HalfCauchy}(10) $$

In [ ]:
def model(x=None, y=None):
  sigma = numpyro.sample("sigma", dist.HalfCauchy(10))
  intercept = numpyro.sample("intercept", dist.Normal(0, 20))
  slope = numpyro.sample("slope", dist.Normal(0, 20))
  likelihood = numpyro.sample("y", dist.Normal(intercept + slope * x, sigma), obs=y)

### MCMCの実行

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=3000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, x=data["x"], y=data["y"])

In [ ]:
mcmc.print_summary()

In [ ]:
idata = az.from_numpyro(mcmc)

In [ ]:
az.plot_trace(idata);

In [ ]:
az.plot_autocorr(idata, combined=True, figsize=(16,3));

### 回帰直線
* 4つのchainのうちの最初のchainの、最初の200個のサンプルに対応する回帰直線を、すべて描いてみる。

In [ ]:
plt.figure(figsize=(7, 5))

n_lines = 200

posterior = idata.posterior

x_axis = np.linspace(0, 1, 11)
intercepts = posterior.intercept.data[0,:n_lines].reshape(1, -1)
slopes = posterior.slope.data[0,:n_lines].reshape(1, -1)
mu_pred = intercepts + slopes * x_axis.reshape(-1, 1)
mu_mean = mu_pred.mean(1)

plt.plot(x, y, "x", label="data")
plt.plot(x, true_regression_line, c="r", label="true regression line", lw=3)
plt.plot(x_axis, mu_pred, color="black", alpha=0.025)
plt.legend(loc=0);

### 予測値の期待値の信用区間
* 説明変数のさまざまな値に対応する目的変数の期待値の信用区間を可視化する。

* まず、切片と傾きの各サンプルを使って、回帰式を計算する。

In [ ]:
intercepts = posterior.intercept.data.flatten().reshape(-1, 1)
slopes = posterior.slope.data.flatten().reshape(-1, 1)
posterior_mu = intercepts + slopes * x.reshape(1, -1)

In [ ]:
posterior_mu.shape

* 期待値の信用区間をHPDIとして求める。

In [ ]:
hpdi_mu = hpdi(posterior_mu, 0.9)

* 人工的に作った200個のデータのxの値それぞれについて、90% highest posterior density interval (HPDI)が求まっている。
 * 転置したほうが見やすいので、転置してから表示している。

In [ ]:
hpdi_mu.T

* 90% HPDIを可視化する。

In [ ]:
plt.figure(figsize=(7, 5))

plt.plot(x, y, "x", label="data")
# Plot recovered linear regression
x_range = np.linspace(0, 1, 2000)
y_pred = posterior.slope.mean().item() * x_range + posterior.intercept.mean().item()
plt.plot(x_range, y_pred, color="black", linestyle="--", label="Recovered regression line")
# Plot 90% CI
plt.fill_between(x, hpdi_mu[0], hpdi_mu[1], alpha=0.3, interpolate=True, label="90% HPDI")
# Plot true regression line
plt.plot(x, true_regression_line, c="r", label="true regression line", lw=3)
plt.legend(loc=0);

## ロバストなベイズ線形回帰
* 観測データが外れ値を含む場合、モデリングを工夫するほうがいい。
* 誤差項が正規分布に従うと仮定してしまうと、回帰直線が外れ値に引っ張られてしまう。
* そこで、**誤差項がt分布に従う**と仮定する。

### 外れ値を含むデータ集合の作成

* 上で作ったデータ集合に、わざと外れ値を追加する。

In [ ]:
x_out = jnp.append(x, jnp.array([0.1, 0.15, 0.2]))
y_out = jnp.append(y, jnp.array([8, 6, 9]))

# xの値でソートしておく。
idx = jnp.argsort(x_out)
x_out = x_out[idx]
y_out = y_out[idx]

data_outlier = dict(x=x_out, y=y_out)

* 真の回帰直線は、変更しない。
 * 外れ値は外れ値なので。

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax = fig.add_subplot(111, xlabel="x", ylabel="y")
ax.plot(x_out, y_out, "x", label="data")
ax.plot(x, true_regression_line, label="true regression line", c="r")
plt.legend(loc=0);

### 誤差項が正規分布に従うと仮定したモデル
* 先ほどと同様、誤差項が正規分布に従うものとしてモデリングしてみる。
* よって、モデルは流用し、観測データだけを入れ替えてMCMCを実行する。

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, x=data_outlier["x"], y=data_outlier["y"])

In [ ]:
mcmc.print_summary()

In [ ]:
idata_normal = az.from_numpyro(mcmc)

* 事後分布について注目すべき点
 * interceptの分布の山が、1より大きい方へずれている。
 * slopeの分布の山が、2より小さい方へずれている。

In [ ]:
az.plot_trace(idata_normal);

In [ ]:
az.plot_autocorr(idata_normal, combined=True, figsize=(16,3));

* 予測値の期待値の信用区間を求める。

In [ ]:
posterior = idata_normal.posterior

intercepts = posterior.intercept.data.flatten().reshape(-1, 1)
slopes = posterior.slope.data.flatten().reshape(-1, 1)
posterior_mu = intercepts + slopes * x_out.reshape(1, -1)

hpdi_mu = hpdi(posterior_mu, 0.9)

* MCMCによって求められた90% HPDIの区間が、外れ値の方に引っ張られている。

In [ ]:
plt.figure(figsize=(7, 5))

plt.plot(x_out, y_out, "x", label="data")
# Plot recovered linear regression
x_range = np.linspace(0, 1, 2000)
y_pred = posterior.slope.mean().item() * x_range + posterior.intercept.mean().item()
plt.plot(x_range, y_pred, color="black", linestyle="--", label="Recovered regression line")
# Plot 90% CI
plt.fill_between(x_out, hpdi_mu[0], hpdi_mu[1], alpha=0.3, interpolate=True, label="90% HPDI")
# Plot true regression line
plt.plot(x, true_regression_line, c="r", label="true regression line", lw=3)
plt.legend(loc=0);

### 誤差項がt分布に従うと仮定したモデル
* t分布は、正規分布よりも、裾野が厚い。
 * 正規分布よりも、外れ値を許容する度合いが高くなる。
* 事前分布の設定方法は、以下の記事を参考にした。
 * https://jrnold.github.io/bayesian_notes/robust-regression.html

* 誤差項が従うt分布のパラメータにも、事前分布を設定する。
* scaleパラメータはhalf Student T分布に従うと仮定する。
 * cf. https://en.wikipedia.org/wiki/Student%27s_t-distribution
 * half Student T分布は、`numpyro.dist.FoldedDistribution`クラスを利用して、自分で定義する。
* t分布の自由度パラメータはガンマ分布に従うと仮定する。
 * cf. https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations


$$ \sigma_0 \sim \text{Half-Cauchy}(10)$$
$$ \nu_0 \sim \text{Gamma}(2, 0.1)$$
$$ \sigma \sim hlst(0, \sigma_0^2, \nu_0)$$
$$ \nu \sim \text{Gamma}(2, 0.1)$$
$$ \beta_0 \sim N(0, 20^2)$$
$$ \beta_1 \sim N(0, 20^2)$$
$$ Y \sim lst(\beta^\top X, \sigma^2, \nu)$$

* 略記の意味は以下の通り。
  * hlst = half location-scale student distribution
  * lst = location-scale student distribution

In [ ]:
def FoldedStudentT(df, loc=0.0, scale=1.0):
  return dist.FoldedDistribution(dist.StudentT(df, loc=loc, scale=scale))

In [ ]:
def model_t(x=None, y=None):
  sigma0 = numpyro.sample("sigma0", dist.HalfCauchy(10))
  nu0 = numpyro.sample("nu0", dist.Gamma(2, 0.1))
  sigma = numpyro.sample("sigma", FoldedStudentT(nu0, scale=sigma0))
  nu = numpyro.sample("nu", dist.Gamma(2, 0.1))
  intercept = numpyro.sample("intercept", dist.Normal(0, 20))
  slope = numpyro.sample("slope", dist.Normal(0, 20))
  likelihood = numpyro.sample("y", dist.StudentT(nu, intercept + slope * x, sigma), obs=y)

In [ ]:
rng_key, rng_key_ = random.split(rng_key)
kernel = NUTS(model_t)
mcmc = MCMC(kernel, num_warmup=1000, num_samples=2000, num_chains=4)
mcmc.run(rng_key_, x=data_outlier["x"], y=data_outlier["y"])

In [ ]:
mcmc.print_summary()

In [ ]:
idata_t = az.from_numpyro(mcmc)

* 傾きと切片について、外れ値を追加する前と、あまり変わらない結果になっている。

In [ ]:
az.plot_trace(idata_t);

In [ ]:
az.plot_autocorr(idata_t, combined=True, figsize=(16,6));

In [ ]:
posterior = idata_t.posterior

intercepts = posterior.intercept.data.flatten().reshape(-1, 1)
slopes = posterior.slope.data.flatten().reshape(-1, 1)
posterior_mu = intercepts + slopes * x_out.reshape(1, -1)

hpdi_mu = hpdi(posterior_mu, 0.9)

In [ ]:
plt.figure(figsize=(7, 5))

plt.plot(x_out, y_out, "x", label="data")
# Plot recovered linear regression
x_range = np.linspace(0, 1, 2000)
y_pred = posterior.slope.mean().item() * x_range + posterior.intercept.mean().item()
plt.plot(x_range, y_pred, color="black", linestyle="--", label="Recovered regression line")
# Plot 90% CI
plt.fill_between(x_out, hpdi_mu[0], hpdi_mu[1], alpha=0.3, interpolate=True, label="90% HPDI")
# Plot true regression line
plt.plot(x, true_regression_line, c="r", label="true regression line", lw=3)
plt.legend(loc=0);

### モデルの比較
* 理論的な説明は、 https://arxiv.org/abs/1507.04544 を参照。

In [ ]:
models = { "gaussian": idata_normal, "Student T": idata_t }
df_compare = az.compare(models)
df_compare

* 上掲論文には、warningがTrueになっている場合、モデルがそもそもロバストでない（一部のデータに強い影響を受けてしまっている）可能性がある、と書いてある。

* 横軸の対数尤度は大きい方が良い。

In [ ]:
az.plot_compare(df_compare);